In [242]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")

In [243]:
# load original dataset

df = pd.read_csv("datasets/data.csv")
df.head()

,Total Kasus,Aceh,Bali,Banten,Babel,Bengkulu,DIY,Jakarta,Jambi,Jabar,...,Riau,Malut,Maluku,Papbar,Papua,Sulbar,NTT,Gorontalo,?,Unnamed: 36
0,18-Mar,0,1,17,0,0,3,158,0,24,...,1,0,0,0,0,0,0,0,0,NaN
1,19-Mar,0,1,27,0,0,5,210,0,26,...,2,0,0,0,0,0,0,0,0,NaN
2,20-Mar,0,4,37,0,0,4,215,0,41,...,1,0,0,0,0,0,0,0,13,NaN
3,21-Mar,0,3,43,0,0,5,267,0,55,...,1,0,0,0,0,0,0,0,10,NaN
4,22-Mar,0,3,47,0,0,5,307,0,59,...,1,0,1,0,2,0,0,0,6,NaN


In [244]:
df[df['Aceh'] == 'Aceh'] # shown 7 different tables that will be used

,Total Kasus,Aceh,Bali,Banten,Babel,Bengkulu,DIY,Jakarta,Jambi,Jabar,...,Riau,Malut,Maluku,Papbar,Papua,Sulbar,NTT,Gorontalo,?,Unnamed: 36
292,Kasus Harian,Aceh,Bali,Banten,Babel,Bengkulu,DIY,Jakarta,Jambi,Jabar,...,Riau,Malut,Maluku,Papbar,Papua,Sulbar,NTT,Gorontalo,?,NaN
588,Kasus Aktif,Aceh,Bali,Banten,Babel,Bengkulu,DIY,Jakarta,Jambi,Jabar,...,Riau,Malut,Maluku,Papbar,Papua,Sulbar,NTT,Gorontalo,?,NaN
878,Sembuh,Aceh,Bali,Banten,Babel,Bengkulu,DIY,Jakarta,Jambi,Jabar,...,Riau,Malut,Maluku,Papbar,Papua,Sulbar,NTT,Gorontalo,?,NaN
1168,Sembuh Harian,Aceh,Bali,Banten,Babel,Bengkulu,DIY,Jakarta,Jambi,Jabar,...,Riau,Malut,Maluku,Papbar,Papua,Sulbar,NTT,Gorontalo,?,NaN
1458,Meninggal Dunia,Aceh,Bali,Banten,Babel,Bengkulu,DIY,Jakarta,Jambi,Jabar,...,Riau,Malut,Maluku,Papbar,Papua,Sulbar,NTT,Gorontalo,?,NaN
1751,Meninggal Dunia Harian,Aceh,Bali,Banten,Babel,Bengkulu,DIY,Jakarta,Jambi,Jabar,...,Riau,Malut,Maluku,Papbar,Papua,Sulbar,NTT,Gorontalo,?,NaN


In [245]:
diff_table = df[df['Aceh'] == 'Aceh'].index # index of different table
diff_table

Int64Index([292, 588, 878, 1168, 1458, 1751], dtype='int64')

In [246]:
exclude = df[df['Aceh'] == 'Jakarta'].index
exclude[0] # index of table that will be excluded

2040

In [247]:
# then we seperate the tables
total_kasus = df.iloc[:diff_table[0]-1].set_index("Total Kasus")
kasus_harian = df.iloc[diff_table[0]+1:diff_table[1]-1].set_index("Total Kasus")
kasus_aktif = df.iloc[diff_table[1]+1:diff_table[2]-1].set_index("Total Kasus")
sembuh = df.iloc[diff_table[2]+1:diff_table[3]-1].set_index("Total Kasus")
sembuh_harian = df.iloc[diff_table[3]+1:diff_table[4]-1].set_index("Total Kasus")
meninggal_dunia = df.iloc[diff_table[4]+1:diff_table[5]-1].set_index("Total Kasus")
meninggal_dunia_harian = df.iloc[diff_table[5]+1:exclude[0]-1].set_index("Total Kasus")

In [248]:
# get dataframe name function
def get_df_name(df):
    name = [x for x in globals() if globals()[x] is df][0]
    return name

In [249]:
list_table = [total_kasus, kasus_harian, kasus_aktif, sembuh,
              sembuh_harian, meninggal_dunia, meninggal_dunia_harian]

# check the number of rows & columns of tables
for i in list_table:
    print(f"{get_df_name(i):<25} contains {i.shape[0]} rows and {i.shape[1]} columns")
    
# from the results, we can see that some of the table has different number of rows
# because of the early covid-19 data hasn't been there.

total_kasus               contains 291 rows and 36 columns
kasus_harian              contains 294 rows and 36 columns
kasus_aktif               contains 288 rows and 36 columns
sembuh                    contains 288 rows and 36 columns
sembuh_harian             contains 288 rows and 36 columns
meninggal_dunia           contains 291 rows and 36 columns
meninggal_dunia_harian    contains 287 rows and 36 columns


### Processing the Data

In [250]:
total_kasus.info() # it shows the data type is not numeric

<class 'pandas.core.frame.DataFrame'>
Index: 291 entries, 18-Mar to 2 Jan
Data columns (total 36 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Aceh         291 non-null    object
 1   Bali         291 non-null    object
 2   Banten       291 non-null    object
 3   Babel        291 non-null    object
 4   Bengkulu     291 non-null    object
 5   DIY          291 non-null    object
 6   Jakarta      291 non-null    object
 7   Jambi        291 non-null    object
 8   Jabar        291 non-null    object
 9   Jateng       291 non-null    object
 10  Jatim        291 non-null    object
 11  Kalbar       291 non-null    object
 12  Kaltim       291 non-null    object
 13  Kalteng      291 non-null    object
 14  Kalsel       291 non-null    object
 15  Kaltara      291 non-null    object
 16  Kep Riau     291 non-null    object
 17  NTB          291 non-null    object
 18  Sumsel       291 non-null    object
 19  Sumbar       291 non-null  

In [251]:
def to_numeric(df):
    col = df.columns
    for i in range(len(col)):
        df[col[i]] = pd.to_numeric(df[col[i]].fillna(0).apply(lambda x: str(x).replace(",", "")))

In [252]:
# drop "Unnamed: 36" column and change data type to integer
# also make a new column: the sum of each row named 'total'
for i in list_table:
    i.drop("Unnamed: 36", axis=1, inplace=True)
    i = to_numeric(i)
    i['total'] = i.sum(axis=1)

### Visualization